# Question 2 Analysis 2
- find top 10 words that frequently occur in top 50 in lead paragraphs of 1993 and not in top 50 of 2016
- if there are not enough 10 words like this, show all words in top 50 of 1993 that not in top 50 of 2016
- the words are not supposed to be stop words or punctuation
- show the result in frequency-descending order in CSV file

In [1]:
import glob
import os
import json
import string
import nltk
from pathlib import Path
from collections import Counter

In [19]:
# use a def to get the count of words in lead paragraph
# the word should not include any punctuations or figures
# the word should not be stop words, which is hardly meaningful for our analysis
def getWordCount(path):
    files = glob.glob(path)
    word_counter = Counter()
    for file in files:
        with open(file, 'r', encoding='utf-8', errors='ignore') as f:
            python_data = json.load(f)
            docs = python_data['response']['docs']
            for doc in docs:
                if 'lead_paragraph' in doc and doc['lead_paragraph'] is not None:
                    for word in map(lambda w: w.strip(string.punctuation).lower(), doc['lead_paragraph'].split()):
                        if word != '' and word not in set(nltk.corpus.stopwords.words('english')) and word.isalpha():
                            word_counter.update([word])
    return word_counter

In [20]:
# prepare the paths to each data set
p = Path(os.getcwd())
ar_1993_path = str(p.parent) + '/data/archive/1993/*'
ar_2016_path = str(p.parent) + '/data/archive/2016/*'

In [21]:
# use the def to get both counters which include the words and their frequency in those lead paragraphs
counter_1993 = getWordCount(ar_1993_path)
counter_2016 = getWordCount(ar_2016_path)
print(len(counter_1993))
print(len(counter_2016))

92255
57629


In [22]:
# we can see from above, there are a lot more different words used in 1993 than in 2016 :D
# iterate collections and find most common words in 1993 which not in 2016
result_list = []
for word,_ in counter_1993.most_common(50):
    if word not in (item[0] for item in counter_2016.most_common(50)):
        result_list.append(word)
print(len(result_list))

18


In [24]:
# there are 18 such words
# store these words into CSV file in frequency-descending order
import csv
path = 'ana_2/disappear_words.csv'
with open(path, 'w') as outcsv:
    writer = csv.DictWriter(outcsv, fieldnames = ["WORD_DISAPPERED"])
    writer.writeheader()
    for word in result_list:
        writer.writerow({'WORD_DISAPPERED': word})
